<h3> Carto 10  sauvegarde des cartes sous forme offsets en binaire uint 16 npy

In [1]:
import os, shutil
import math
import folium
import numpy as np
import sys
import torch
import math
import time

from scipy.ndimage import label, generate_binary_structure

from cachetools import TTLCache
from threading import Lock
from fonctions2024 import *
from fonctions2025 import *
from collections import defaultdict, deque     # necessaire pour la carte 

In [2]:
# ! pip install torch


In [3]:


# def charge_masque_simple(y0,x0):
#     resolution=1
#     lat=math.ceil(y0)
#     lng=math.floor(x0)
#     pas=730
#     resolution=1
   
#     # on cherche si le fichier existe dans les txt
#     file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
#     # print(basedir)
#     filelocal='carto/'+file
#     print ('filename',filelocal)
#     if os.path.exists(filelocal) == True:                                             # si le fichier existe on le charge
#         with open(filelocal, "rb") as fichier:
#             cartetexte=fichier.read()
           
#             print('Le fichier carte a ete recuperé en local')
#         # carte=eval(cartetexte)  
#         # print(carte)

#     else: 
#         print('pas de recuperation du fichier en local')                                            # si le fichier n'existe pas on va le chercher chez vr
#         lng_f=str(int(lng/10))
#         lat_f=str(int(lat/10))
#         folder='/'+str(resolution)+'/'+lng_f+'/'+lat_f+'/'
#         file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
#         folderlocal='carto/'
#         filelocal=folderlocal+file
#         #filelocal=file
        
#         url='https://static.virtualregatta.com/ressources/maps/dalles/vro2k16'+folder+file
#         print('url',url)
#         headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" }
#         response = requests.get(url, headers=headers)
#         print (response.status_code)
#         #print (response.content)


#         if response.status_code == 200:
#             print('fichierlocal ',filelocal)
#             with open(filelocal, "wb") as file:
#                 file.write(response.content)
#                 print("Fichier téléchargé avec succès.")
           
#         else:
#             print(f"Erreur {response.status_code} : Impossible de récupérer le fichier chez vr .")

#         #2) Ouverture du fichier VR et extraction du masque
#     with open(filelocal,'rb') as fid:
#         header=fid.read(11)
#         gzbuf=fid.read()
#         databuf=zlib.decompress(gzbuf,-zlib.MAX_WBITS)
#     data=np.frombuffer(databuf,dtype=np.int8)
#     data.resize((730,730))
#     mask=   ( data > -1) *1

#         # #3) Conversion du masque en polygone
#         # carte=extrait_terres(lat,lng,pas,mask)

#         # #4 on le sauvegarde en txt et en js local
#         # # ecriture dans fichier
#         # lng_f=str(int(lng/10))
#         # lat_f=str(int(lat/10))
#         # folder=str(resolution)+'/'+lng_f+'/'+lat_f+'/'
#         # file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.txt'
#         # filename='/home/jp/staticLocal/carto/txt/'+file
#         # # print(' sauvegarde du txt ',filename)
#         # textecarte=str(carte)
#         # with open(filename, "w+") as fichier:
#         #     fichier.write(textecarte)
   
     
#     return mask   

# # for i in range (30,40):
# #     for j in range (-10,0):
# #         charge_masque_simple(i,j)
        
# #         print ('masque charge ',i,j)
        
# # y0=45
# # x0=-3
# # mask=charge_masque_simple(y0,x0)

In [2]:
def charge_masque_simple(y0,x0):
    resolution=1
    lat=math.ceil(y0)
    lng=math.floor(x0)
    pas=730
    resolution=1
   
    # on cherche si le fichier existe dans les txt
    file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
    # print(basedir)
    filelocal='carto/'+file
    # print ('filename',filelocal)
    if os.path.exists(filelocal) == True:                                             # si le fichier existe on le charge
        with open(filelocal, "rb") as fichier:
            cartetexte=fichier.read()
           
            print('Le fichier {} a ete recuperé en local' .format (filelocal))
        # carte=eval(cartetexte)  
        # print(carte)

    else: 
        # print('pas de recuperation du fichier en local')                                            # si le fichier n'existe pas on va le chercher chez vr
        lng_f=str(int(lng/10))
        lat_f=str(int(lat/10))
        folder='/'+str(resolution)+'/'+lng_f+'/'+lat_f+'/'
        file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
        folderlocal='carto/'
        filelocal=folderlocal+file
        #filelocal=file
        
        url='https://static.virtualregatta.com/ressources/maps/dalles/vro2k16'+folder+file
        print('url',url)
        headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" }
        response = requests.get(url, headers=headers)
        print (response.status_code)
        #print (response.content)


        if response.status_code == 200:
            # print('fichierlocal ',filelocal)
            with open(filelocal, "wb") as file:
                file.write(response.content)
                print('Le fichier {} a ete telecharge' .format (filelocal))
                # print("Fichier téléchargé avec succès.")
           
        else:
            print(f"Erreur {response.status_code} : Impossible de récupérer le fichier chez vr .")

        #2) Ouverture du fichier VR et extraction du masque quel que soit la methode de recup
  
    with open(filelocal,'rb') as fid:
        header=fid.read(11)
        gzbuf=fid.read()
        databuf=zlib.decompress(gzbuf,-zlib.MAX_WBITS)
    data=np.frombuffer(databuf,dtype=np.int8)
    data.resize((730,730))
    mask=   ( data > -1) *1

    # masque unitaire pour y0 x0 
    return mask   


        

In [3]:
def chargeMasque (y0,x0):
    resolution=1
    lat=math.ceil(y0)
    lng=math.floor(x0)
    pas=730
    file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
    # print(basedir)
    filename='carto/'+file
    with open(filename,'rb') as fid:
            header=fid.read(11)
            gzbuf=fid.read()
            databuf=zlib.decompress(gzbuf,-zlib.MAX_WBITS)
    data=np.frombuffer(databuf,dtype=np.int8)
    data.resize((730,730))
    mask=   ( data > -1) 
    return mask 

In [4]:


def assemble_masques(lat_min, lat_max, lon_min, lon_max):
    latitudes = range(math.floor(lat_max), math.ceil(lat_min), -1)
    longitudes = range(math.floor(lon_min), math.ceil(lon_max))
    
    masques = []
    for lat in latitudes:
        ligne = []
        for lon in longitudes:
            try:
                mask = chargeMasque(lat, lon)
            except Exception as e:
                print(f"Erreur en chargeant masque {lat},{lon} : {e}")
                mask = np.zeros((730,730), dtype=np.uint8)
            ligne.append(mask)
        masques.append(np.concatenate(ligne, axis=1))
    masque_global = np.concatenate(masques, axis=0)
    
    return masque_global, latitudes[0], longitudes[0]


In [5]:
# def retirer_lacs(masque):
#     """
#     Enlève les lacs intérieurs d’un masque booléen (True=mer, False=terre),
#     en gardant uniquement la mer connectée au bord.
#     """
#     # Connexité 4 (segments H/V seulement, pas diagonales)
#     masque = ~masque 
#     structure = generate_binary_structure(2, 1)

#     # Labelisation des zones d'eau (True = mer)
#     labels, num_labels = label(masque, structure=structure)

#     # On récupère les labels présents sur les bords du masque
#     bords = np.concatenate([
#         labels[0, :],           # bord haut
#         labels[-1, :],          # bord bas
#         labels[:, 0],           # bord gauche
#         labels[:, -1]           # bord droit
#     ])
#     labels_mer_connectee = np.unique(bords)
#     labels_mer_connectee = labels_mer_connectee[labels_mer_connectee != 0]

#     # On crée un nouveau masque avec uniquement les zones d'eau connectées au bord
#     masque_sans_lacs = np.isin(labels, labels_mer_connectee)

#     return masque_sans_lacs


In [6]:


def detecter_segments_numpy(masque):
    nrows, ncols = masque.shape

    # Discontinuités verticales (entre lignes) → segments horizontaux
    diff_vert = masque[:-1, :] != masque[1:, :]  # shape (nrows-1, ncols)
    i_vert, j_vert = np.where(diff_vert)
    # (x1,y1,x2,y2) pour segments horizontaux
    seg_h = np.stack([j_vert, i_vert + 1, j_vert + 1, i_vert + 1], axis=1)

    # Discontinuités horizontales (entre colonnes) → segments verticaux
    diff_horz = masque[:, :-1] != masque[:, 1:]  # shape (nrows, ncols-1)
    i_horz, j_horz = np.where(diff_horz)
    # (x1,y1,x2,y2) pour segments verticaux
    seg_v = np.stack([j_horz + 1, i_horz, j_horz + 1, i_horz + 1], axis=1)

    return seg_h, seg_v


def regrouper_segments_alignes_numpy(segments, axe_fixe='y'):
    """
    Fusionne segments colinéaires (x1,y1,x2,y2) qui se suivent.
    axe_fixe = 'y' pour segments horizontaux (y fixe), 'x' pour verticaux (x fixe)
    Retourne tableau fusionné de mêmes segments.
    """

    if len(segments) == 0:
        return segments

    # Trier segments pour fusion (par axe fixe, puis coord variable)
    if axe_fixe == 'y':
        # segments horizontaux : trier par y1, puis x1
        idx_sort = np.lexsort((segments[:, 0], segments[:, 1]))
    else:
        # segments verticaux : trier par x1, puis y1
        idx_sort = np.lexsort((segments[:, 1], segments[:, 0]))

    segments = segments[idx_sort]

    # Liste pour segments fusionnés
    fusion = []
    debut = segments[0].copy()

    for seg in segments[1:]:
        if axe_fixe == 'y':
            # même y et la fin du précédent est la début du suivant en x
            if (seg[1] == debut[1]) and (seg[0] == debut[2]):
                # étendre le segment fusionné
                debut[2] = seg[2]
                debut[3] = seg[3]
            else:
                fusion.append(debut)
                debut = seg.copy()
        else:
            # même x et fin précédent == début suivant en y
            if (seg[0] == debut[0]) and (seg[1] == debut[3]):
                debut[2] = seg[2]
                debut[3] = seg[3]
            else:
                fusion.append(debut)
                debut = seg.copy()
    fusion.append(debut)

    return np.array(fusion)


def segments_to_coords_folium_numpy(segments, lat0, lon0, pas):
    """
    Convertit segments numpy (x1,y1,x2,y2) en liste pour folium : [[[lat1, lon1], [lat2, lon2]], ...]
    """
    x1 = segments[:, 0]
    y1 = segments[:, 1]
    x2 = segments[:, 2]
    y2 = segments[:, 3]

    lat1 = lat0 - y1 * pas
    lon1 = lon0 + x1 * pas
    lat2 = lat0 - y2 * pas
    lon2 = lon0 + x2 * pas

    coords = [ [[lat1[i], lon1[i]], [lat2[i], lon2[i]]] for i in range(len(segments)) ]
    return coords


def construire_carte(masque, lat0, lon0, pas):
    seg_h, seg_v = detecter_segments_numpy(masque)

    # Fusion segments horizontaux (axe y fixe)
    seg_h_fus = regrouper_segments_alignes_numpy(seg_h, 'y')
    # Fusion segments verticaux (axe x fixe)
    seg_v_fus = regrouper_segments_alignes_numpy(seg_v, 'x')

    # Conversion en coords geographiques
    coords_h = segments_to_coords_folium_numpy(seg_h_fus, lat0, lon0, pas)
    coords_v = segments_to_coords_folium_numpy(seg_v_fus, lat0, lon0, pas)

    # Fusion finale
    return coords_h + coords_v



In [7]:
def chargeMasque (y0,x0):
    resolution=1
    lat=math.ceil(y0)
    lng=math.floor(x0)
    pas=730
    file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
    # print(basedir)
    filename='carto/'+file
    with open(filename,'rb') as fid:
            header=fid.read(11)
            gzbuf=fid.read()
            databuf=zlib.decompress(gzbuf,-zlib.MAX_WBITS)
    data=np.frombuffer(databuf,dtype=np.int8)
    data.resize((730,730))
    mask=   ( data > -1) 
    return mask 

<h3> Version en float 32

In [8]:
# En chainement des taches 
tic =time.time()
pas=1/730
# Definition de la tuile par son angle superieur gauche
lat0=60
lon0=0



lat_min=lat0-10
lat_max=lat0
lon_min=lon0
lon_max=lon0+10

for i in range (lat0-10,lat0+1):
    for j in range (lon0,lon0+10+1):
        charge_masque_simple(i,j)        
print ('\nmasques charges entre lat {} et {} et lon entre {} et {}'.format(lat0-10,lat0,lon0,lon0+10))

# On assemble les masques 
masquecomplet, lat0, lon0 =assemble_masques(lat_min, lat_max, lon_min, lon_max)
print ('\nmasques assembles entre lat {} et {} et lon entre {} et {}'.format(lat0-10,lat0,lon0,lon0+10))

# on retire les rivieres et les lacs
masque=(retirer_lacs(masquecomplet))

print ('Les lacs on ete retires du masque global')
print ('shape ',masque.shape)
print (masque)
# on construit la carte 
carte=construire_carte(masque, lat0, lon0, pas)



# print ('carte',carte)
nomcarte='carte_'+str(lat0)+'_'+str(lon0)+'.npy'
print (nomcarte)
filename =nomcarte
np.save(nomcarte, np.array(carte, dtype=np.float32))
# print (carte)

print ('temps exe ',time.time()-tic)

carte = np.load(filename)

# test de la carte 
tic=time.time()
print (filename)
carte = np.load(filename)
m = folium.Map([lat0,lon0] , zoom_start=10)
folium.LatLngPopup().add_to(m)   # popup lat long
try:
    folium.PolyLine(carte, color='blue', weight=1).add_to(m)
except:
    print ('la carte est vide' )
    folium.PolyLine([[lat0,lon0],[lat0,lon0+10],[lat0-10,lon0+10],[lat0-10,lon0],[lat0,lon0]],color='blue', weight=1).add_to(m)
print ('temps tracage ',time.time()-tic)    
m

# on vide le repertoire fichier carto
# dossier = "/home/jp/pytorch/carto/"

# for nom in os.listdir(dossier):
#     chemin = os.path.join(dossier, nom)
#     if os.path.isfile(chemin) or os.path.islink(chemin):
#         os.unlink(chemin)      # supprime fichier ou lien
#     elif os.path.isdir(chemin):
#         shutil.rmtree(chemin)  # supprime dossier

# print('temps affichage ',time.time()-tic)

Le fichier carto/1_0_50.deg a ete recuperé en local
Le fichier carto/1_1_50.deg a ete recuperé en local
Le fichier carto/1_2_50.deg a ete recuperé en local
Le fichier carto/1_3_50.deg a ete recuperé en local
Le fichier carto/1_4_50.deg a ete recuperé en local
Le fichier carto/1_5_50.deg a ete recuperé en local
Le fichier carto/1_6_50.deg a ete recuperé en local
Le fichier carto/1_7_50.deg a ete recuperé en local
Le fichier carto/1_8_50.deg a ete recuperé en local
Le fichier carto/1_9_50.deg a ete recuperé en local
Le fichier carto/1_10_50.deg a ete recuperé en local
Le fichier carto/1_0_51.deg a ete recuperé en local
Le fichier carto/1_1_51.deg a ete recuperé en local
Le fichier carto/1_2_51.deg a ete recuperé en local
Le fichier carto/1_3_51.deg a ete recuperé en local
Le fichier carto/1_4_51.deg a ete recuperé en local
Le fichier carto/1_5_51.deg a ete recuperé en local
Le fichier carto/1_6_51.deg a ete recuperé en local
Le fichier carto/1_7_51.deg a ete recuperé en local
Le fichier 

NameError: name 'retirer_lacs' is not defined

<h3> Chargement et tracage a partir d 'un point quelconque

In [9]:
#on va charger une carte a partir de lat et lon et l afficher 
lat=29.15
lon=140
# determination de la carte
latAngle=int(10*(lat//10) +10)
lonAngle=int(10*(lon//10))
print (latAngle,lonAngle)
filename='maps/carte_'+str(latAngle)+'_'+str(lonAngle)+'.npy'
carte = np.load(filename)


m = folium.Map([lat,lon] , zoom_start=10)
folium.LatLngPopup().add_to(m)   # popup lat long
folium.Marker([lat,lon], color='blue').add_to(m)

try:
    folium.PolyLine(carte, color='blue', weight=1).add_to(m)
except:
    print ('la carte est vide' )
    folium.PolyLine([[lat0,lon0],[lat0,lon0+10],[lat0-10,lon0+10],[lat0-10,lon0],[lat0,lon0]],color='blue', weight=1).add_to(m)
print ('temps tracage ',time.time()-tic)    
m



30 140


FileNotFoundError: [Errno 2] No such file or directory: 'maps/carte_30_140.npy'

<h3> Version en uint16

In [10]:

# def charge_masque_simple(y0,x0):
#     resolution=1
#     lat=math.ceil(y0)
#     lng=math.floor(x0)
#     pas=730
#     resolution=1
   
#     # on cherche si le fichier existe dans les txt
#     file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
#     # print(basedir)
#     filelocal='carto/'+file
#     print ('filename',filelocal)
#     if os.path.exists(filelocal) == True:                                             # si le fichier existe on le charge
#         with open(filelocal, "rb") as fichier:
#             cartetexte=fichier.read()
           
#             print('Le fichier carte a ete recuperé en local')
#         # carte=eval(cartetexte)  
#         # print(carte)

#     else: 
#         print('pas de recuperation du fichier en local')                                            # si le fichier n'existe pas on va le chercher chez vr
#         lng_f=str(int(lng/10))
#         lat_f=str(int(lat/10))
#         folder='/'+str(resolution)+'/'+lng_f+'/'+lat_f+'/'
#         file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
#         folderlocal='carto/'
#         filelocal=folderlocal+file
#         #filelocal=file
        
#         url='https://static.virtualregatta.com/ressources/maps/dalles/vro2k16'+folder+file
#         print('url',url)
#         headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" }
#         response = requests.get(url, headers=headers)
#         print (response.status_code)
#         #print (response.content)


#         if response.status_code == 200:
#             print('fichierlocal ',filelocal)
#             with open(filelocal, "wb") as file:
#                 file.write(response.content)
#                 print("Fichier téléchargé avec succès.")
           
#         else:
#             print(f"Erreur {response.status_code} : Impossible de récupérer le fichier chez vr .")

#         #2) Ouverture du fichier VR et extraction du masque
#     with open(filelocal,'rb') as fid:
#         header=fid.read(11)
#         gzbuf=fid.read()
#         databuf=zlib.decompress(gzbuf,-zlib.MAX_WBITS)
#     data=np.frombuffer(databuf,dtype=np.int8)
#     data.resize((730,730))
#     mask=   ( data > -1) *1

#         # #3) Conversion du masque en polygone
#         # carte=extrait_terres(lat,lng,pas,mask)

#         # #4 on le sauvegarde en txt et en js local
#         # # ecriture dans fichier
#         # lng_f=str(int(lng/10))
#         # lat_f=str(int(lat/10))
#         # folder=str(resolution)+'/'+lng_f+'/'+lat_f+'/'
#         # file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.txt'
#         # filename='/home/jp/staticLocal/carto/txt/'+file
#         # # print(' sauvegarde du txt ',filename)
#         # textecarte=str(carte)
#         # with open(filename, "w+") as fichier:
#         #     fichier.write(textecarte)
   
     
#     return mask   


def retirer_lacs(masque):
    """
    Enlève les lacs intérieurs d’un masque booléen (True=mer, False=terre),
    en gardant uniquement la mer connectée au bord.
    """
    # Connexité 4 (segments H/V seulement, pas diagonales)
    masque = ~masque 
    structure = generate_binary_structure(2, 1)

    # Labelisation des zones d'eau (True = mer)
    labels, num_labels = label(masque, structure=structure)

    # On récupère les labels présents sur les bords du masque
    bords = np.concatenate([
        labels[0, :],           # bord haut
        labels[-1, :],          # bord bas
        labels[:, 0],           # bord gauche
        labels[:, -1]           # bord droit
    ])
    labels_mer_connectee = np.unique(bords)
    labels_mer_connectee = labels_mer_connectee[labels_mer_connectee != 0]

    # On crée un nouveau masque avec uniquement les zones d'eau connectées au bord
    masque_sans_lacs = np.isin(labels, labels_mer_connectee)

    return masque_sans_lacs


def detecter_segments_numpy(masque):
    nrows, ncols = masque.shape

    # Discontinuités verticales (entre lignes) → segments horizontaux
    diff_vert = masque[:-1, :] != masque[1:, :]  # shape (nrows-1, ncols)
    i_vert, j_vert = np.where(diff_vert)
    # (x1,y1,x2,y2) pour segments horizontaux
    seg_h = np.stack([j_vert, i_vert + 1, j_vert + 1, i_vert + 1], axis=1)

    # Discontinuités horizontales (entre colonnes) → segments verticaux
    diff_horz = masque[:, :-1] != masque[:, 1:]  # shape (nrows, ncols-1)
    i_horz, j_horz = np.where(diff_horz)
    # (x1,y1,x2,y2) pour segments verticaux
    seg_v = np.stack([j_horz + 1, i_horz, j_horz + 1, i_horz + 1], axis=1)

    return seg_h, seg_v


def regrouper_segments_alignes_numpy(segments, axe_fixe='y'):
    """
    Fusionne segments colinéaires (x1,y1,x2,y2) qui se suivent.
    axe_fixe = 'y' pour segments horizontaux (y fixe), 'x' pour verticaux (x fixe)
    Retourne tableau fusionné de mêmes segments.
    """

    if len(segments) == 0:
        return segments

    # Trier segments pour fusion (par axe fixe, puis coord variable)
    if axe_fixe == 'y':
        # segments horizontaux : trier par y1, puis x1
        idx_sort = np.lexsort((segments[:, 0], segments[:, 1]))
    else:
        # segments verticaux : trier par x1, puis y1
        idx_sort = np.lexsort((segments[:, 1], segments[:, 0]))

    segments = segments[idx_sort]

    # Liste pour segments fusionnés
    fusion = []
    debut = segments[0].copy()

    for seg in segments[1:]:
        if axe_fixe == 'y':
            # même y et la fin du précédent est la début du suivant en x
            if (seg[1] == debut[1]) and (seg[0] == debut[2]):
                # étendre le segment fusionné
                debut[2] = seg[2]
                debut[3] = seg[3]
            else:
                fusion.append(debut)
                debut = seg.copy()
        else:
            # même x et fin précédent == début suivant en y
            if (seg[0] == debut[0]) and (seg[1] == debut[3]):
                debut[2] = seg[2]
                debut[3] = seg[3]
            else:
                fusion.append(debut)
                debut = seg.copy()
    fusion.append(debut)

    return np.array(fusion)


def segments_to_save_numpy(segments):
    """
    Convertit segments numpy (x1,y1,x2,y2) en liste pour folium : [[[lat1, lon1], [lat2, lon2]], ...]
    """
    x1 = segments[:, 0]
    y1 = segments[:, 1]
    x2 = segments[:, 2]
    y2 = segments[:, 3]

    coords = [ [[y1[i], x1[i]], [y2[i], x2[i]]] for i in range(len(segments)) ]
    return coords

def construire_carte_offset(masque):
    seg_h, seg_v = detecter_segments_numpy(masque)

    # Fusion segments horizontaux (axe y fixe)
    seg_h_fus = regrouper_segments_alignes_numpy(seg_h, 'y')
    # Fusion segments verticaux (axe x fixe)
    seg_v_fus = regrouper_segments_alignes_numpy(seg_v, 'x')

    # Conversion en coords geographiques
    coords_h = segments_to_save_numpy(seg_h_fus)
    coords_v = segments_to_save_numpy(seg_v_fus)

    # Fusion finale
    return coords_h + coords_v

<h3> Chargement des cartes avec boucle

In [12]:
# En chainement des taches 
tic =time.time()
pas=1/730
# Definition de la tuile par son angle superieur gauche
lat0=90
lon0=-90

def charge_fichier(lat0,lon0):

    lat_min=lat0-10
    lat_max=lat0
    lon_min=lon0
    lon_max=lon0+10
    
    for i in range (lat0-10,lat0+1):
        for j in range (lon0,lon0+10+1):
            charge_masque_simple(i,j)        
    print ('\nmasques charges entre lat {} et {} et lon entre {} et {}'.format(lat0-10,lat0,lon0,lon0+10))
    
    # On assemble les masques 
    masquecomplet, lat0, lon0 =assemble_masques(lat_min, lat_max, lon_min, lon_max)
    print ('\nmasques assembles entre lat {} et {} et lon entre {} et {}'.format(lat0-10,lat0,lon0,lon0+10))
    
    
    masque=(retirer_lacs(masquecomplet))  # on retire les rivieres et les lacs
    
    # print ('Les lacs on ete retires du masque global')
    # print ('shape ',masque.shape)
    
    
    offsets=construire_carte_offset(masque)   # on construit la carte des offsets
    filename='carteoffset_'+str(lat0)+'_'+str(lon0)+'.npy'
    
    np.save(filename, np.array(offsets, dtype=np.uint16))  # on va sauver en numpy int16 
    print ( 'fichier {} sauve '.format(filename))
    return None 


# charge_fichier (lat0,-110)
for lon0 in range (110,180,10):
    charge_fichier(lat0,lon0)



Le fichier carto/1_110_80.deg a ete recuperé en local
Le fichier carto/1_111_80.deg a ete recuperé en local
Le fichier carto/1_112_80.deg a ete recuperé en local
Le fichier carto/1_113_80.deg a ete recuperé en local
Le fichier carto/1_114_80.deg a ete recuperé en local
Le fichier carto/1_115_80.deg a ete recuperé en local
Le fichier carto/1_116_80.deg a ete recuperé en local
Le fichier carto/1_117_80.deg a ete recuperé en local
Le fichier carto/1_118_80.deg a ete recuperé en local
Le fichier carto/1_119_80.deg a ete recuperé en local
Le fichier carto/1_120_80.deg a ete recuperé en local
Le fichier carto/1_110_81.deg a ete recuperé en local
Le fichier carto/1_111_81.deg a ete recuperé en local
Le fichier carto/1_112_81.deg a ete recuperé en local
Le fichier carto/1_113_81.deg a ete recuperé en local
Le fichier carto/1_114_81.deg a ete recuperé en local
Le fichier carto/1_115_81.deg a ete recuperé en local
Le fichier carto/1_116_81.deg a ete recuperé en local
Le fichier carto/1_117_81.de

In [ ]:
lat=39.5
lon=141
# determination de la carte
lat0=int(10*(lat//10) +10)
lon0=int(10*(lon//10))
print (lat0,lon0)

filename='carteoffset_'+str(lat0)+'_'+str(lon0)+'.npy'

print ('filename ',filename)


# filename='carteoffset_'+str(lat0)+'_'+str(lon0)+'.npy'
# on recharge le numpy pour voir que la sauvegarde est correcte
offsets = np.load(filename)                                # matrice des offsets

# scale = 1/730                                              # un pas = 1/730 °
# on va transformer le fichier numpy d'offsets en coordonnees 
coords = np.empty_like(offsets, dtype=np.float32)
coords[..., 0] = lat0 - offsets[..., 0].astype(np.float32) * pas    # latitude diminue vers le sud
coords[..., 1] = lon0 + offsets[..., 1].astype(np.float32) * pas    # longitude augmente vers l’est


m = folium.Map([lat0,lon0] , zoom_start=10)
folium.LatLngPopup().add_to(m)   # popup lat long
folium.Marker([lat0,lon0], color='blue').add_to(m)
tic =time.time()
try:
    folium.PolyLine(coords, color='blue', weight=1).add_to(m)
except:
    print ('la carte est vide' )
    folium.PolyLine([[lat0,lon0],[lat0,lon0+10],[lat0-10,lon0+10],[lat0-10,lon0],[lat0,lon0]],color='blue', weight=1).add_to(m)
print ('temps tracage ',time.time()-tic)    
m


In [ ]:
pas=1/730
m = folium.Map([lat0,lon0] , zoom_start=10)
folium.LatLngPopup().add_to(m)   # popup lat long
folium.Marker([lat0,lon0], color='blue').add_to(m)
tic =time.time()
lat0=60
for lon0 in range (-180,0,10):
    try:
        filename='carteoffset_'+str(lat0)+'_'+str(lon0)+'.npy'
        offsets = np.load(filename)                                # matrice des offsets

        # scale = 1/730                                              # un pas = 1/730 °
        # on va transformer le fichier numpy d'offsets en coordonnees 
        coords = np.empty_like(offsets, dtype=np.float32)
        coords[..., 0] = lat0 - offsets[..., 0].astype(np.float32) * pas    # latitude diminue vers le sud
        coords[..., 1] = lon0 + offsets[..., 1].astype(np.float32) * pas    # longitude augmente vers l’est

    
   
        folium.PolyLine(coords, color='blue', weight=1).add_to(m)
        folium.PolyLine([[lat0,lon0],[lat0,lon0+10],[lat0-10,lon0+10],[lat0-10,lon0],[lat0,lon0]],color='blue', weight=1).add_to(m)
    except:
        print ('la carte est vide' )
        folium.PolyLine([[lat0,lon0],[lat0,lon0+10],[lat0-10,lon0+10],[lat0-10,lon0],[lat0,lon0]],color='blue', weight=1).add_to(m)
print ('temps tracage ',time.time()-tic)    
m

In [15]:
cache_cartes = TTLCache(maxsize=1000, ttl=5 * 24 * 60 * 60)
cache_lock = Lock()

In [16]:
# lat=36
# lon=-9
pas=1/730
tic=time.time()

# La carte 10x10 est reperee par son angle superieur gauche soit latmax  et lon min 

lat0=30
lonO=-10


latmax=int(lat0)
latmin=latmax-10
lonmin=int(lon0)
lonmax=lonmin+10


# Construction du masque global
masquecomplet, lat0, lon0 = assemble_masques(latmin,latmax,lonmin,lonmax)
# print (masquecomplet)
masque=(retirer_lacs(masquecomplet))

print (masque)




Erreur en chargeant masque 30,1 : [Errno 2] No such file or directory: 'carto/1_1_30.deg'
Erreur en chargeant masque 30,2 : [Errno 2] No such file or directory: 'carto/1_2_30.deg'
Erreur en chargeant masque 30,3 : [Errno 2] No such file or directory: 'carto/1_3_30.deg'
Erreur en chargeant masque 30,4 : [Errno 2] No such file or directory: 'carto/1_4_30.deg'
Erreur en chargeant masque 30,5 : [Errno 2] No such file or directory: 'carto/1_5_30.deg'
Erreur en chargeant masque 30,6 : [Errno 2] No such file or directory: 'carto/1_6_30.deg'
Erreur en chargeant masque 30,7 : [Errno 2] No such file or directory: 'carto/1_7_30.deg'
Erreur en chargeant masque 30,8 : [Errno 2] No such file or directory: 'carto/1_8_30.deg'
Erreur en chargeant masque 30,9 : [Errno 2] No such file or directory: 'carto/1_9_30.deg'
Erreur en chargeant masque 29,1 : [Errno 2] No such file or directory: 'carto/1_1_29.deg'
Erreur en chargeant masque 29,2 : [Errno 2] No such file or directory: 'carto/1_2_29.deg'
Erreur en 

In [17]:
tic=time.time()
# carte = construire_carte(masque, lat0, lon0, pas)


# print('temps calcul ',time.time()-tic)

tic=time.time()
m = folium.Map([50,-3] , zoom_start=10)
folium.LatLngPopup().add_to(m)   # popup lat long
folium.PolyLine(carte, color='blue', weight=1).add_to(m)
print('temps affichage ',time.time()-tic)
# folium.PolyLine(AB, color='red', weight=5).add_to(m)
# folium.PolyLine([A,C], color='blue', weight=1).add_to(m)
# folium.PolyLine([B,C], color='green', weight=1).add_to(m)
# folium.PolyLine([A,D], color='purple', weight=1).add_to(m)

m  # affiche dans notebook Jupyter

temps affichage  0.3363039493560791


In [18]:
# operation complete 
# 1) on charge les masques 



In [ ]:
A=[ 49.7263,-1.9495]
B=[ 49.7271, -1.9312]
C= (49.7309,-1.9388)
D=(49.9050, -2.0657)
y0,x0=A
y1,x1=B
y2,x2=C
y3,x3=D
AB=[A,B]

In [ ]:
def segments_se_croisent(A, B, C, D):
    """
    Teste si les segments [A,B] et [C,D] se croisent
    A, B, C, D : [lat, lon]
    """
    def ccw(P, Q, R):
        return (R[1] - P[1]) * (Q[0] - P[0]) > (Q[1] - P[1]) * (R[0] - P[0])
    return (ccw(A, C, D) != ccw(B, C, D)) and (ccw(A, B, C) != ccw(A, B, D))


def intersection(segment, carte, marge=0.1):
    [[lat0, lon0], [lat1, lon1]] = segment

    # Fenêtre élargie autour du segment
    latmin = min(lat0, lat1) - marge
    latmax = max(lat0, lat1) + marge
    lonmin = min(lon0, lon1) - marge
    lonmax = max(lon0, lon1) + marge

    A, B = segment

    for C, D in carte:
        # Si le segment côtier est hors de la zone élargie, on saute
        if max(C[0], D[0]) < latmin or min(C[0], D[0]) > latmax:
            continue
        if max(C[1], D[1]) < lonmin or min(C[1], D[1]) > lonmax:
            continue

        if segments_se_croisent(A, B, C, D):
            return True

    return False
    

In [ ]:
segment_test = [A,B]
tic=time.time()
res= intersection(segment_test, carte)
#     print("Segment coupe la terre")
# else:
#     print("Segment en mer")
print ('temps calcul pour 1000',1000*(time.time()-tic))

# segment_test = [B,C]
# if intersection(segment_test, carte):
#     print("Segment coupe la terre")
# else:
#     print("Segment en mer")

# segment_test = [A,C]
# if intersection(segment_test, carte):
#     print("Segment coupe la terre")
# else:
#     print("Segment en mer")

# segment_test = [A,D]
# if intersection(segment_test, carte):
#     print("Segment coupe la terre")
# else:
#     print("Segment en mer")

In [ ]:
routage=torch.tensor([[47.7689,-4.3210],[47.7702,-4.3262],[47.7726,-4.3403],[47.7760,-4.3574],[47.7819,-4.3731],[47.7856,-4.3801],[47.7889,-4.3859],[47.7955,-4.3944],[47.7982,-4.39780],[47.8020,-4.4065]],device='cuda')
routage_np = routage.cpu().numpy()
# print(routage_np)


In [ ]:
map = folium.Map(location=A, zoom_start=8)
folium.LatLngPopup().add_to(map)   # popup lat long
folium.PolyLine(carte  , color='blue', weight=1).add_to(map)
folium.PolyLine(routage, color='red' , weight=1).add_to(map)
for i in range (len(routage_np)-1):
    segment=[[routage_np[i,0],routage_np[i,1]] , [routage_np[i+1,0],routage_np[i+1,1]]]           
    if intersection(segment,carte):
        couleur='red'
    else:
        couleur='blue'
    
    folium.PolyLine(segment, color=couleur, weight=1).add_to(map)
for i in range (len(routage_np)):
    
   folium.Circle(routage_np[i], color='black',radius=10, weight=1).add_to(map) 
map

In [ ]:
# fonction globale de chargement d 'une carte 10 / 10
y=45.5
x=2
# on arrondit modulo 
print(y//10*10+5)
print(x//10*10-5)
#on va charger la carte complete correspondant 
lat0=y//10*10+5
lon0=x//10*10-5
def fcarte2(lat,lon):
    ''' construit la carte -5 +5 pour lat et lon '''
    latmin=5 * math.floor(lat/ 5)
    latmax=latmin+10
    lonmin=5 * math.floor(lon / 5)
    lonmax=lonmin+10
    print('latmin',latmin)
    print(lonmin)
    pas=1/730
    masquecomplet, lat0, lon0 = assemble_masques(latmin,latmax,lonmin,lonmax)
    masque=retirer_lacs(masquecomplet)
    return construire_carte(masque, lat0, lon0, pas)

carte2=fcarte2(y,x)

print(len(carte2))

In [ ]:
y=49

x=-0.6
tic=time.time()
carte2=fcarte2(y,x)
print('temps de calcul= ',time.time()-tic)
map = folium.Map(Location=[y,x], zoom_start=8)
folium.LatLngPopup().add_to(map)   # popup lat long
tic=time.time()
folium.PolyLine(carte2  , color='blue', weight=1).add_to(map)
print('temps affichage= ',time.time()-tic)  
folium.Circle([y,x], color='red',radius=1000, weight=2).add_to(map) 
map

In [ ]:
# def retirer_lacs_terre_true(masque_terre_true):
#     """
#     Supprime les lacs (zones d'eau intérieures), en supposant que
#     True = terre, False = mer (ta convention).
#     """
#     masque_inversé = ~masque_terre_true
#     masque_mer_sans_lacs = retirer_lacs(masque_inversé)
#     return ~masque_mer_sans_lacs  # retransformé en terre=True

In [ ]:
# #Automatisation +2-2 autour d'un point
# lat=51
# lon=3

# tic=time.time()
# carte2=fcarte2(y,x)
# print('temps de calcul carte2= ',time.time()-tic)

# tic=time.time()
# latmin=int(lat-2)
# latmax=int(lat+2)

# lonmin=int(lon-4)
# lonmax=int(lon+2)
# # Construction du masque global
# masque_global, lat0, lon0 = assemble_masques(latmin,latmax,lonmin,lonmax)
# resolutionlat=abs(latmax-latmin)
# resolutionlon=abs(lonmax-lonmin) 
# print(resolutionlat)
# print(resolutionlon)
# segments = contour_terre3(masque_global, lat0, lon0,resolutionlat,resolutionlon)
# polylignes = chaine_segments(segments)
# print ('temps de calcul carte1', time.time()-tic)

# carte = folium.Map(location=[lat0 - 1, lon0 + 1], zoom_start=8)
# tic=time.time()
# folium.PolyLine(polylignes, color='blue', weight=1).add_to(carte)
# folium.Circle([lat,lon],radius=100,color='red').add_to(carte)
# print ('temps d affichage carte1', time.time()-tic)
# tic=time.time()
# folium.PolyLine(carte2  , color='red', weight=1).add_to(carte)
# print('temps affichage carte2  rouge = ',time.time()-tic)

# carte


In [ ]:
def get_carte(lat, lon):
    # Centre sur les multiples de 5 (carte 10x10)
    lat_centre = (int(lat) // 5) * 5
    lon_centre = (int(lon) // 5) * 5

    cle = (lat_centre, lon_centre)

    with cache_lock:
        if cle in cache_cartes:
            return cache_cartes[cle]

    # Calcul ou chargement de la carte (taille fixe de 10x10)
    carte = fcarte2(lat_centre, lon_centre)

    with cache_lock:
        cache_cartes[cle] = carte

    return carte

In [ ]:
lat=49
lon=-1.5
tic=time.time()
tic=time.time()
get_carte(lat, lon)
print ('temps de chargement ', time.time()-tic)

In [ ]:
lat=49
lon=-6.5
tic=time.time()
carte2=get_carte(lat, lon)
print ('temps de chargement ', time.time()-tic)

map = folium.Map(Location=[y,x], zoom_start=8)
folium.LatLngPopup().add_to(map)   # popup lat long
tic=time.time()
folium.PolyLine(carte2  , color='blue', weight=1).add_to(map)
print('temps affichage= ',time.time()-tic)  
folium.Circle([y,x], color='red',radius=1000, weight=2).add_to(map) 
map

In [ ]:
# Vient en doublon due la dfonction precedente 

def chargeMasque (y0,x0):
    # charge un masque a partir du fichier local s il existe 
    resolution=1
    lat=math.ceil(y0)
    lng=math.floor(x0)
    pas=730
    file=str(resolution)+'_'+str(lng)+'_'+str(lat)+'.deg'
    # print(basedir)
    filename='carto/'+file
    with open(filename,'rb') as fid:
            header=fid.read(11)
            gzbuf=fid.read()
            databuf=zlib.decompress(gzbuf,-zlib.MAX_WBITS)
    data=np.frombuffer(databuf,dtype=np.int8)
    data.resize((730,730))
    mask=   ( data > -1) 
    return mask 
